In [10]:
import pandas as pd

# 1. Analysing samples of dataframes with df.groupby().__iter__()

It’s usually hard to explore a dataset row by row or group by group within a Jupyter Notebook compared to what you can do with Excel. One useful trick is to use a generator and use Ctrl + Enter instead of Shift + Enter in order to iteratively look at different samples within the same cell, without creating a mess in your notebook.

First create a cell with the generator with .groupby() (or .iterrows()) and add the .__iter__():

In [2]:
# generator = df.groupby(['identifier']).__iter__()

Then, run the following cell as many times as you wish to observe the data that matters most to you, using the keyboard shortcut Ctrl + Enter:

In [3]:
# group_id, grouped_data = generator.__next__()
# print(group_id) 
# grouped_data

In [4]:
df = pd.read_csv("P:/titanicdataset-traincsv/train.csv")

In [5]:
# Create a list of tickets that are shared ba more than 2 passengers
tickets_with_mote_than_2_passangers = df["Ticket"].value_counts().loc[
    (df["Ticket"].value_counts() > 2)
].index.tolist()

In [6]:
# Crate generator to loop througt this tickets
generator = df.loc[df["Ticket"].isin(tickets_with_mote_than_2_passangers)].groupby("Ticket").__iter__()

In [7]:
# Look at this tickets individually, using Ctrl+Enter
group_id, grouped_data = generator.__next__()
print(group_id)
grouped_data

110152


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
257,258,1,1,"Cherry, Miss. Gladys",female,30.0,0,0,110152,86.5,B77,S
504,505,1,1,"Maioni, Miss. Roberta",female,16.0,0,0,110152,86.5,B79,S
759,760,1,1,"Rothes, the Countess. of (Lucy Noel Martha Dye...",female,33.0,0,0,110152,86.5,B77,S


# 2. Pandas Profiling for data exploration and data quality assessment

As often in data science, we tend to write from scratch our data exploratory code. Since all datasets are different, it makes sense. However, there’s the magical pandas_profiling package that makes this reasoning pointless. The package actually automates the data exploration and data quality assessment steps!

In [7]:
import pandas_profiling 
report = pandas_profiling.ProfileReport(df)

C:\Users\Root\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [8]:
report.to_notebook_iframe()

AttributeError: 'ProfileReport' object has no attribute 'to_notebook_iframe'

In [15]:
df.profile_report()

In [18]:
profile = df.profile_report(title="Pandas Profiling Report")
profile.to_file(output_file="Pandas Profiling Report — AirBNB .html")

# 3. Multi chaining

Pandas becomes really fun once you understand that you can combine multiple operations using the chaining method. Chaining is basically adding operations within the same ‘line’ of code.

With the line of code below, I am
- adding a new column in my dataset (.merge)
- counting the proportion of female passengers (.apply(female_proportion))
- for groups of more than 1 passengers (df.Ticket.value_counts()>1)
- having the same ticket number (.groupby(‘Ticket’)).

In [19]:
def female_proportion(dataframe):
    return (dataframe.Sex=='female').sum() / len(dataframe)

female_proportion(df)

0.35241301907968575

In [20]:
df.merge(
    df.loc[
        df.Ticket.isin(
            df.Ticket.value_counts().loc[
                df.Ticket.value_counts()>1
            ].index
        )
    ].groupby('Ticket').apply(female_proportion) \
     .reset_index().rename(columns={0:'proportion_female'}),
     how='left', on='Ticket'
)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,proportion_female
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,NaN
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.5
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,NaN
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,NaN
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0.5
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,NaN


# 4. Plotting coefficients / Feature Importance with style

If you’re doing Machine Learning, you know the struggle to explain clearly your Machine Learning model in layman’s terms. One way to help here is to have a good visual of the coefficients or the variable importances within your model.


I’ve gathered code from multiple sources to get the following visualisation that I re-use all the time (and avoid reinventing the wheel):

In [22]:
pd.DataFrame({
    'variable': variables,
    'coefficient': model.coef_[0]
}) \
    .round(decimals=2) \
    .sort_values('coefficient', ascending=False) \
    .style.bar(color=['grey', 'lightblue'], align='zero')

NameError: name 'variables' is not defined

# 5. sklearn_pandas

If you’re a pandas advocate, you have come to realise more than once that working with pandas DataFrame and sklearn isn’t always the best fit. But don’t stop here. A handful of motivated contributors have created sklearn_pandas, the bridge between the two packages. It replaces sklearn’s ColumnTransformer with a pandas-friendly DataFrameMapper. I’ve started using only sklearn_pandas these days and I’m not disappointed. I wish I had known this before.

In [23]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn_pandas import DataFrameMapper
from category_encoders import LeaveOneOutEncoder

imputer_Pclass = SimpleImputer(strategy='most_frequent', add_indicator=True)
imputer_Age = SimpleImputer(strategy='median', add_indicator=True)
imputer_SibSp = SimpleImputer(strategy='constant', fill_value=0, add_indicator=True)
imputer_Parch = SimpleImputer(strategy='constant', fill_value=0, add_indicator=True)
imputer_Fare = SimpleImputer(strategy='median', add_indicator=True)
imputer_Embarked = SimpleImputer(strategy='most_frequent')

scaler_Age = MinMaxScaler()
scaler_Fare = StandardScaler()

onehotencoder_Sex = OneHotEncoder(drop=['male'], handle_unknown='error')
onehotencoder_Embarked = OneHotEncoder(handle_unknown='error')

leaveoneout_encoder = LeaveOneOutEncoder(sigma=.1, random_state=2020)

mapper = DataFrameMapper([
    (['Age'], [imputer_Age, scaler_Age], {'alias':'Age_scaled'}),
    (['Pclass'], [imputer_Pclass]),
    (['SibSp'], [imputer_SibSp]),
    (['Parch'], [imputer_Parch]),
    (['Fare'], [imputer_Fare, scaler_Fare], {'alias': 'Fare_scaled'}),
    (['Sex'], [onehotencoder_Sex], {'alias': 'is_female'}),
    (['Embarked'], [imputer_Embarked, onehotencoder_Embarked]), 
    (['Embarked_Pclass_Sex'], [leaveoneout_encoder])
], df_out=True) # use df_out to output as a pandas DataFrame

mapper.fit(X=train, y=train['Survived']) # you fit it like a sklearn ColumnTransformer

ModuleNotFoundError: No module named 'sklearn_pandas'

# 6. tqdm

When dealing with large datasets, data manipulation will take time. Instead of waiting with a dull face in front of your Jupyter Notebook without knowing what’s happening, use tqdm to track whether your code is actually running and how long it takes. It’s also a good way to abort early on a script that is too slow to run.

In [24]:
from tqdm import notebook
notebook.tqdm().pandas()

Now all pandas DataFrame have new methods:
- .progress_apply, .progress_applymap
- .progress_map for columns

They’re the same as apply, applymap and map with the difference that they’ll plot a progress bar. How useful!

# 7. the .to_clipboard() to paste in Excel

One thing that I ended up doing a lot to cope with that is to export my results to Excel. But not using the .to_excel method. Instead, I use the smoother .to_clipboard(index=False) that copies data to the clipboard. I then use Ctrl + V in Excel to paste the data in my current spreadsheet. And here you go, you may unleash the Excel beast that’s inside you.

One thing that a lot of data scientists tend to forget is that non data scientists are generally skilled in Excel. It’s easier to share an Excel file with them rather than a Notebook or a Notebook exported as an html file.